In [32]:
from numba import njit,types
from numba.typed import List
from collections import Counter

In [52]:
@njit()
def board_is_done(bins):
    for b in bins:
        if not bin_is_done(b, empty_is_done=True):
            return False
    return True

@njit()
def get_top_ball(bi):
    for ball in bi[::-1]:
        if ball > -1:
            return ball
    return -1

@njit()
def pop_top_ball(bi):
    for i,ball in enumerate(bi[::-1]):
        if ball > -1:
            bi[3-i] = -1
            return ball
    return -1

@njit()
def set_top_ball(bi,b):
    for i,ball in enumerate(bi):
        if ball == -1:
            bi[i] = b
            break

@njit()
def bin_is_done(bi, empty_is_done=False):
    if empty_is_done:
        return (bi[0] == bi[1]) and (bi[1] == bi[2]) and (bi[2] == bi[3])
    else:
        return (bi[0] != -1) and (bi[0] == bi[1]) and (bi[1] == bi[2]) and (bi[2] == bi[3])

@njit()
def do_move(bins,i:int,j:int):
    b = pop_top_ball(bins[i])
    set_top_ball(bins[j],b)

@njit()
def get_available_moves(bins):

    num_bins = len(bins)
    moves = List()

    for from_i in range(num_bins):

        if bin_is_done(bins[from_i]):
            continue

        for to_i in range(num_bins):

            if from_i == to_i:
                continue

            from_b = get_top_ball(bins[from_i])
            to_b = get_top_ball(bins[to_i])

            if from_b == -1:
                continue
            elif to_b == -1:
                moves.append((from_i,to_i))
            elif from_b == to_b:
                moves.append((from_i,to_i))
    
    return moves

In [59]:
initial_bins = [[1,2,3,3],[1,4,5,5],[2,2,2,-1],[6,7,7,7],[1,8,8,8],
                [9,9,-1,-1],[-1,-1,-1,-1],[9,4,10,6],[3,5,5,6],[9,8,4,4],
                [10,6,11,11],[11,12,10,10],[11,7,3,1],[12,12,12,-1],[13,13,13,13]]

In [60]:
Counter(ba for bi in initial_bins for ba in bi)

Counter({1: 4,
         2: 4,
         3: 4,
         4: 4,
         5: 4,
         -1: 8,
         6: 4,
         7: 4,
         8: 4,
         9: 4,
         10: 4,
         11: 4,
         12: 4,
         13: 4})

In [61]:
initial_bins_nb = List()

for bi in initial_bins:
    temp = List(lsttype=types.ListType(types.int16))
    for ba in bi:
        temp.append(ba)
    initial_bins_nb.append(temp)

print(initial_bins_nb)

[[1, 2, 3, 3, ...], [1, 4, 5, 5, ...], [2, 2, 2, -1, ...], [6, 7, 7, 7, ...], [1, 8, 8, 8, ...], [9, 9, -1, -1, ...], [-1, -1, -1, -1, ...], [9, 4, 10, 6, ...], [3, 5, 5, 6, ...], [9, 8, 4, 4, ...], [10, 6, 11, 11, ...], [11, 12, 10, 10, ...], [11, 7, 3, 1, ...], [12, 12, 12, -1, ...], [13, 13, 13, 13, ...], ...]


In [62]:
def one2one(bins):
    return tuple(sorted(map(tuple,bins)))

In [67]:
seen = {}
best_sol = [None]*100

def recurse(move_seq, bin_state):
    
    global best_sol
    
    depth = len(move_seq)
    if depth >= len(best_sol):
        return
    if depth > 50:
        return
    
    bin_state_hash = one2one(bin_state)
    if bin_state_hash in seen and seen[bin_state_hash] <= depth:
        return
    
    seen[bin_state_hash] = depth
    
    if board_is_done(bin_state):
        print(f'found sol at depth {depth}, seen size = {len(seen)}')
        if len(move_seq) < len(best_sol):
            best_sol = move_seq.copy()
        return
    
    possible_moves = get_available_moves(bin_state)
    
    if depth > 0:
        reverse_prev_move = (move_seq[-1][1], move_seq[-1][0])
        if reverse_prev_move in possible_moves:
            possible_moves.remove(reverse_prev_move)
    
    for possible_move in possible_moves:
        do_move(bin_state,possible_move[0],possible_move[1])
        move_seq.append(possible_move)
        recurse(move_seq,bin_state)
        move_seq.pop()
        do_move(bin_state,possible_move[1],possible_move[0])

In [68]:
move_seq = []

recurse(move_seq,initial_bins_nb)

In [71]:
for bins,depth in seen.items():
    print(depth)
    print(bins)
    print('-'*30)

0
((1, -1, -1, -1), (2, 2, 2, 2), (4, 4, -1, -1), (5, -1, -1, -1), (5, 5, -1, -1), (6, 6, -1, -1), (7, 7, 7, -1), (9, -1, -1, -1), (9, 8, 8, 8), (9, 9, -1, -1), (10, -1, -1, -1), (11, 3, 3, 3), (11, 10, -1, -1), (12, 12, 12, -1), (13, 13, 13, 13))
------------------------------
1
((-1, -1, -1, -1), (1, -1, -1, -1), (2, 2, 2, 2), (4, 4, -1, -1), (5, -1, -1, -1), (5, 5, -1, -1), (6, 6, -1, -1), (7, 7, 7, -1), (9, 8, 8, 8), (9, 9, 9, -1), (10, -1, -1, -1), (11, 3, 3, 3), (11, 10, -1, -1), (12, 12, 12, -1), (13, 13, 13, 13))
------------------------------
2
((-1, -1, -1, -1), (-1, -1, -1, -1), (1, -1, -1, -1), (2, 2, 2, 2), (4, 4, -1, -1), (5, 5, 5, -1), (6, 6, -1, -1), (7, 7, 7, -1), (9, 8, 8, 8), (9, 9, 9, -1), (10, -1, -1, -1), (11, 3, 3, 3), (11, 10, -1, -1), (12, 12, 12, -1), (13, 13, 13, 13))
------------------------------
3
((-1, -1, -1, -1), (1, -1, -1, -1), (2, 2, 2, 2), (4, 4, -1, -1), (5, 5, 5, -1), (6, 6, -1, -1), (7, -1, -1, -1), (7, 7, -1, -1), (9, 8, 8, 8), (9, 9, 9, -1), (1

In [69]:
best_sol

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]